In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report

# Constants
NASNET_SIZE = (331, 331)  # NASNetLarge required size
BATCH_SIZE = 16
EPOCHS = 50
DATA_DIR = '/kaggle/input/images-skinlesion/images'
CLASS_NAMES = ['MEL', 'NV', 'BCC', 'SCC']

def preprocess_input(x):
    # Apply NASNetLarge preprocessing
    return tf.keras.applications.nasnet.preprocess_input(x)

# Enhanced data augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect',
    brightness_range=[0.7, 1.3],
    channel_shift_range=0.2,
    validation_split=0.2
)

# Create generators with NASNET_SIZE
train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=NASNET_SIZE,  # Using NASNet required size directly
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    classes=CLASS_NAMES,
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=NASNET_SIZE,  # Using NASNet required size directly
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    classes=CLASS_NAMES,
    shuffle=False
)

# Calculate class weights
total_samples = 8200
class_weights = {
    0: total_samples/(1000 * len(CLASS_NAMES)),  # MEL
    1: total_samples/(6000 * len(CLASS_NAMES)),  # NV
    2: total_samples/(600 * len(CLASS_NAMES)),   # BCC
    3: total_samples/(600 * len(CLASS_NAMES))    # SCC
}

# Create model
base_model = NASNetLarge(
    weights='imagenet',
    include_top=False,
    input_shape=(*NASNET_SIZE, 3)
)

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = Dropout(0.3)(x)
outputs = Dense(len(CLASS_NAMES), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)

# Initialize with frozen layers
for layer in base_model.layers:
    layer.trainable = False

# Compile model
optimizer = Adam(learning_rate=0.0001)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)



Found 7174 images belonging to 4 classes.
Found 1791 images belonging to 4 classes.


In [4]:
# Advanced callbacks
callbacks = [
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        '/kaggle/working/best_model_nasnet.keras',
        save_best_only=True,
        monitor='val_accuracy',
        mode='max',
        verbose=1
    )
]

# Phase 1: Train top layers
print("Phase 1: Training top layers...")
history1 = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=10,
    class_weight=class_weights,
    callbacks=callbacks
)

# Phase 2: Fine-tune entire model
print("Phase 2: Fine-tuning entire model...")
for layer in model.layers:
    layer.trainable = True

# Recompile with lower learning rate for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

# Full training
history2 = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=callbacks,
    initial_epoch=10
)



Phase 1: Training top layers...
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1728042491.488718     112 service.cc:145] XLA service 0x7bc71c003670 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728042491.488771     112 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1728042491.488776     112 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1728042519.903814     112 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 636ms/step - accuracy: 0.4897 - auc_1: 0.7432 - loss: 23.3374
Epoch 1: val_accuracy improved from -inf to 0.61486, saving model to /kaggle/working/best_model_nasnet.keras
448/448 ━━━━━━━━━━━━━━━━━━━━ 435s 834ms/step - accuracy: 0.4898 - auc_1: 0.7434 - loss: 23.3349 - val_accuracy: 0.6149 - val_auc_1: 0.8462 - val_loss: 20.2290 - learning_rate: 1.0000e-04
Epoch 2/10
  1/448 ━━━━━━━━━━━━━━━━━━━━ 3:09 424ms/step - accuracy: 0.5625 - auc_1: 0.7806 - loss: 20.1097

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_accuracy did not improve from 0.61486
448/448 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.5625 - auc_1: 0.7806 - loss: 20.1097 - val_accuracy: 0.1333 - val_auc_1: 0.6644 - val_loss: 20.8253 - learning_rate: 1.0000e-04
Epoch 3/10
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 527ms/step - accuracy: 0.6297 - auc_1: 0.8669 - loss: 19.3512
Epoch 3: val_accuracy improved from 0.61486 to 0.66104, saving model to /kaggle/working/best_model_nasnet.keras
448/448 ━━━━━━━━━━━━━━━━━━━━ 305s 672ms/step - accuracy: 0.6297 - auc_1: 0.8668 - loss: 19.3494 - val_accuracy: 0.6610 - val_auc_1: 0.8864 - val_loss: 16.9543 - learning_rate: 1.0000e-04
Epoch 4/10
  1/448 ━━━━━━━━━━━━━━━━━━━━ 3:04 414ms/step - accuracy: 0.6875 - auc_1: 0.8672 - loss: 17.5031
Epoch 4: val_accuracy did not improve from 0.66104
448/448 ━━━━━━━━━━━━━━━━━━━━ 1s 874us/step - accuracy: 0.6875 - auc_1: 0.8672 - loss: 17.5031 - val_accuracy: 0.1333 - val_auc_1: 0.5548 - val_loss: 17.9893 - learning_rate: 1.0000e-04
Epoch 5/10
448/4

I0000 00:00:1728044879.390784     115 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_add_reduce_fusion_3', 4 bytes spill stores, 4 bytes spill loads



448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6028 - auc_2: 0.8536 - loss: 9.9573
Epoch 11: val_accuracy improved from 0.67399 to 0.77421, saving model to /kaggle/working/best_model_nasnet.keras
448/448 ━━━━━━━━━━━━━━━━━━━━ 1310s 2s/step - accuracy: 0.6028 - auc_2: 0.8536 - loss: 9.9571 - val_accuracy: 0.7742 - val_auc_2: 0.9099 - val_loss: 10.3244 - learning_rate: 1.0000e-05
Epoch 12/50
  1/448 ━━━━━━━━━━━━━━━━━━━━ 10:48 1s/step - accuracy: 0.6250 - auc_2: 0.8548 - loss: 9.4499
Epoch 12: val_accuracy did not improve from 0.77421
448/448 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - accuracy: 0.6250 - auc_2: 0.8548 - loss: 9.4499 - val_accuracy: 0.0667 - val_auc_2: 0.2015 - val_loss: 20.4304 - learning_rate: 1.0000e-05
Epoch 13/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6471 - auc_2: 0.8825 - loss: 9.5299
Epoch 13: val_accuracy improved from 0.77421 to 0.77872, saving model to /kaggle/working/best_model_nasnet.keras
448/448 ━━━━━━━━━━━━━━━━━━━━ 700s 2s/step - accuracy: 0.6

In [5]:
# Evaluation and visualization functions
def plot_training_history(history1, history2):
    plt.figure(figsize=(15, 5))
    
    acc = history1.history['accuracy'] + history2.history['accuracy']
    val_acc = history1.history['val_accuracy'] + history2.history['val_accuracy']
    loss = history1.history['loss'] + history2.history['loss']
    val_loss = history1.history['val_loss'] + history2.history['val_loss']
    
    plt.subplot(1, 2, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('/kaggle/working/training_history_nasnet.png')
    plt.close()

# Generate predictions
predictions = model.predict(validation_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = validation_generator.classes

# Plot confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=CLASS_NAMES, 
            yticklabels=CLASS_NAMES)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('/kaggle/working/confusion_matrix_nasnet.png')
plt.close()

# Print classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES))

# Save final model
model.save('/kaggle/working/skin_lesion_classifier_nasnet_final.keras')

# Plot training history
plot_training_history(history1, history2)

print("Training completed. Model and visualization artifacts saved.")

112/112 ━━━━━━━━━━━━━━━━━━━━ 88s 651ms/step

Classification Report:
              precision    recall  f1-score   support

         MEL       0.67      0.66      0.67       222
          NV       0.93      0.95      0.94      1341
         BCC       0.59      0.89      0.71       102
         SCC       0.86      0.35      0.50       126

    accuracy                           0.87      1791
   macro avg       0.76      0.71      0.70      1791
weighted avg       0.88      0.87      0.86      1791

Training completed. Model and visualization artifacts saved.
